### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import heapq


### Reading CSV files of data

In [313]:
user_info = pd.read_csv("C:/Users/DELL/Documents/Hackathon/Our_User_Info.csv")
user_rating = pd.read_csv("C:/Users/DELL/Documents/Hackathon/rating_1to5.csv")
rest_rating = pd.read_csv("C:/Users/DELL/Documents/Hackathon/Our_Rest_Info.csv")

user_updated = user_info.copy()
user_updated = user_updated.drop("ModePayment", axis=1)

user_updated['Fast_Food'] = np.nan
user_updated['Chinese'] = np.nan
user_updated['Desi'] = np.nan
user_updated['Regional'] = np.nan
user_updated['Dessert-Ice_Cream'] = np.nan

user_rating['Rating'] = user_rating[['rating','food_rating','service_rating']].mean(axis=1)

user_updated.head()

,ID,Cuisine,Longitude,Latitude,Age,Budget,Fast_Food,Chinese,Desi,Regional,Dessert-Ice_Cream
0,1001,Fast_Food,-100.978803,22.139997,30,medium,NaN,NaN,NaN,NaN,NaN
1,1002,Fast_Food,-100.983325,22.150087,29,low,NaN,NaN,NaN,NaN,NaN
2,1003,Fast_Food,-100.946527,22.119847,30,low,NaN,NaN,NaN,NaN,NaN
3,1004,Fast_Food,-99.183000,18.867000,79,medium,NaN,NaN,NaN,NaN,NaN
4,1005,Fast_Food,-100.959891,22.183477,27,medium,NaN,NaN,NaN,NaN,NaN


## Age ranges, budget to int, cuisine to int

In [106]:
age = {range(20, 30): 1, range(30, 40): 2, range(40, 55): 3, range(55, 100): 4}

user_updated['Age'] = user_updated['Age'].apply(lambda x: next((v for k, v in age.items() if x in k), 0))
user_updated.head()

budget = {'low' : 1, 'medium' : 2, 'high' : 3, '?': 0 }
user_updated['Budget'] = user_updated['Budget'].apply(lambda x: budget[x])

user_updated.head()

,ID,Cuisine,Longitude,Latitude,Age,Budget,Fast_Food,Chinese,Desi,Regional,Dessert-Ice_Cream
0,1001,Fast_Food,-100.978803,22.139997,2,2,NaN,NaN,NaN,NaN,NaN
1,1002,Fast_Food,-100.983325,22.150087,1,1,NaN,NaN,NaN,NaN,NaN
2,1003,Fast_Food,-100.946527,22.119847,2,1,NaN,NaN,NaN,NaN,NaN
3,1004,Fast_Food,-99.183000,18.867000,4,2,NaN,NaN,NaN,NaN,NaN
4,1005,Fast_Food,-100.959891,22.183477,1,2,NaN,NaN,NaN,NaN,NaN


In [107]:
count = 0
for IDs in user_updated['ID']:
    list1 = (user_updated['Cuisine'].get(count)).split(",")
    
    if 'Fast_Food' in list1:
        user_updated['Fast_Food'][count] = 1
    else:
        user_updated['Fast_Food'][count] = 0
    
    if 'Chinese' in list1:
        user_updated['Chinese'][count] = 1
    else:
        user_updated['Chinese'][count] = 0
    
    if 'Desi' in list1:
        user_updated['Desi'][count] = 1
    else:
        user_updated['Desi'][count] = 0
        
    if 'Regional' in list1:
        user_updated['Regional'][count] = 1
    else:
        user_updated['Regional'][count] = 0
    
    if 'Dessert-Ice_Cream' in list1:
        user_updated['Dessert-Ice_Cream'][count] = 1
    else:
        user_updated['Dessert-Ice_Cream'][count] = 0
    
    count += 1


E:\Software_Installations\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Software_Installations\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
E:\Software_Installations\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Software_Installations\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWar

In [112]:
user_updated = user_updated.drop("Cuisine", axis=1)
user_updated = user_updated.drop("ID", axis=1)
user_updated = user_updated.drop("Longitude", axis=1)
user_updated = user_updated.drop("Latitude", axis=1)

user_updated.to_csv(r'C:/Users/DELL/Documents/Hackathon/user_updated.csv', index=False)
user_updated.head()

,Age,Budget,Fast_Food,Chinese,Desi,Regional,Dessert-Ice_Cream
0,2,2,1.0,0.0,0.0,0.0,0.0
1,1,1,1.0,0.0,0.0,0.0,0.0
2,2,1,1.0,0.0,0.0,0.0,0.0
3,4,2,1.0,0.0,0.0,0.0,0.0
4,1,2,1.0,0.0,0.0,0.0,0.0


In [123]:
user_matrix = pd.DataFrame(columns=['Budget', 'Fast_Food', 'Chinese', 'Desi', 'Regional', 'Dessert-Ice_Cream'])

user_updated = pd.read_csv("C:/Users/DELL/Documents/Hackathon/user_updated.csv")

user_matrix['Budget'] = user_updated['Budget']
user_matrix['Fast_Food'] = user_updated['Fast_Food']
user_matrix['Chinese'] = user_updated['Chinese']
user_matrix['Desi'] = user_updated['Desi']
user_matrix['Regional'] = user_updated['Regional']
user_matrix['Dessert-Ice_Cream'] = user_updated['Dessert-Ice_Cream']
user_matrix.head()

,Budget,Fast_Food,Chinese,Desi,Regional,Dessert-Ice_Cream
0,2,1.0,0.0,0.0,0.0,0.0
1,1,1.0,0.0,0.0,0.0,0.0
2,1,1.0,0.0,0.0,0.0,0.0
3,2,1.0,0.0,0.0,0.0,0.0
4,2,1.0,0.0,0.0,0.0,0.0


### Find Cosine Similarity Matrix and find the indices of similar user

In [238]:
similarity_matrix = cosine_similarity(user_matrix)
similarity_matrix[similarity_matrix >= 0.98] = 0
topn = 5

user_similar_array = np.zeros((len(user_info['ID']),5))
user_index_array = []

for index in range(len(user_info['ID'])):
    user_similar_array[index] = heapq.nlargest(topn, similarity_matrix[index])

for index in range(len(user_info['ID'])):
    temp = np.in1d(similarity_matrix[index], user_similar_array[index])
    temp2 = [i for i, x in enumerate(temp) if x]
    if (len(temp2) > 5):
        user_index_array.append(temp2[len(temp2)-5:len(temp2)])
    else:
        user_index_array.append(temp2)
    

### Map indexes to user_IDs  

In [239]:
mapper_idx = {
    i: ID for i, ID in 
    enumerate(list(user_info['ID'].values))
}
mapper_idx

{0: 1001,
 1: 1002,
 2: 1003,
 3: 1004,
 4: 1005,
 5: 1006,
 6: 1007,
 7: 1008,
 8: 1009,
 9: 1010,
 10: 1011,
 11: 1012,
 12: 1013,
 13: 1014,
 14: 1015,
 15: 1016,
 16: 1017,
 17: 1018,
 18: 1019,
 19: 1020,
 20: 1021,
 21: 1022,
 22: 1023,
 23: 1024,
 24: 1025,
 25: 1026,
 26: 1027,
 27: 1028,
 28: 1029,
 29: 1030,
 30: 1031,
 31: 1032,
 32: 1033,
 33: 1034,
 34: 1035,
 35: 1036,
 36: 1037,
 37: 1038,
 38: 1039,
 39: 1040,
 40: 1041,
 41: 1042,
 42: 1043,
 43: 1044,
 44: 1045,
 45: 1046,
 46: 1047,
 47: 1048,
 48: 1049,
 49: 1050,
 50: 1051,
 51: 1052,
 52: 1053,
 53: 1054,
 54: 1055,
 55: 1056,
 56: 1057,
 57: 1058,
 58: 1059,
 59: 1060,
 60: 1061,
 61: 1062,
 62: 1063,
 63: 1064,
 64: 1065,
 65: 1066,
 66: 1067,
 67: 1068,
 68: 1069,
 69: 1070,
 70: 1071,
 71: 1072,
 72: 1073,
 73: 1074,
 74: 1075,
 75: 1076,
 76: 1077,
 77: 1078,
 78: 1079,
 79: 1080,
 80: 1081,
 81: 1082,
 82: 1083,
 83: 1084,
 84: 1085,
 85: 1086,
 86: 1087,
 87: 1088,
 88: 1089,
 89: 1090,
 90: 1091,
 91: 1092

In [240]:
def matching_ID(mapper, fav):

    """
    return the ID of a user based on its index
    
    Parameters
    ----------    
    mapper: dict, map placeID to the index of that restaurant in the data
    fav: str, ID of the desired user
    
    Return
    ------
    index against an ID
    
    """
    
    match_tuple = []
    for i, idx in mapper.items():
        if i == fav:
            match_tuple.append((i, idx))

    if not match_tuple:
        print('Oops! No match is found')
        return None

    return match_tuple[0][1]
matching_ID(mapper_idx, 69)

1070

### Covert indices from Similarity index array to unique user IDs

In [241]:
# temp
for i in range(len(user_index_array)):
    for j in range(5):
        user_index_array[i][j] = matching_ID(mapper_idx, user_index_array[i][j])
#         print(user_index_array[i][j])
    
user_index_array

[[1070, 1080, 1081, 1089, 1091],
 [1098, 1100, 1103, 1104, 1105],
 [1098, 1100, 1103, 1104, 1105],
 [1070, 1080, 1081, 1089, 1091],
 [1070, 1080, 1081, 1089, 1091],
 [1070, 1080, 1081, 1089, 1091],
 [1098, 1100, 1103, 1104, 1105],
 [1081, 1089, 1091, 1101, 1108],
 [1118, 1119, 1121, 1123, 1138],
 [1070, 1080, 1081, 1089, 1091],
 [1070, 1080, 1081, 1089, 1091],
 [1042, 1045, 1071, 1099, 1107],
 [1070, 1080, 1081, 1089, 1091],
 [1070, 1080, 1081, 1089, 1091],
 [1070, 1080, 1081, 1089, 1091],
 [1118, 1119, 1121, 1123, 1138],
 [1125, 1126, 1127, 1128, 1129],
 [1098, 1100, 1103, 1104, 1105],
 [1070, 1080, 1081, 1089, 1091],
 [1070, 1080, 1081, 1089, 1091],
 [1118, 1119, 1121, 1123, 1138],
 [1070, 1080, 1081, 1089, 1091],
 [1098, 1100, 1103, 1104, 1105],
 [1125, 1126, 1127, 1128, 1129],
 [1070, 1080, 1081, 1089, 1091],
 [1070, 1080, 1081, 1089, 1091],
 [1098, 1100, 1103, 1104, 1105],
 [1070, 1080, 1081, 1089, 1091],
 [1098, 1100, 1103, 1104, 1105],
 [1070, 1080, 1081, 1089, 1091],
 [1070, 10

In [264]:
user_similarity = pd.DataFrame(columns=['ID'])
user_similarity['ID'] = user_info['ID']

user_similarity = pd.concat([user_similarity, pd.DataFrame(user_index_array)], axis=1)



,ID,0,1,2,3,4
0,1001,1070,1080,1081,1089,1091
1,1002,1098,1100,1103,1104,1105
2,1003,1098,1100,1103,1104,1105
3,1004,1070,1080,1081,1089,1091
4,1005,1070,1080,1081,1089,1091
5,1006,1070,1080,1081,1089,1091
6,1007,1098,1100,1103,1104,1105
7,1008,1081,1089,1091,1101,1108
8,1009,1118,1119,1121,1123,1138
9,1010,1070,1080,1081,1089,1091


In [370]:
def make_recommendation(similarity_data, rating, user_ID, n_recommendations, unique_users, rest_info):
    
    """
    return the recommended restaurants against a restaurant
    
    Parameters
    ----------    
    similarity_data: dataframe, cosine similarity matrix
    rating: dataframe, rating dataframe of a user for 
    user_ID: str, ID of the user
    n_recommendations: int, total number of restaurants needed for that particular restaurants ID
    
    Return
    ------
    index against an ID
    
    """
    similar_users = np.zeros((n_recommendations,1))

    for i in range(unique_users):
        if (similarity_data['ID'].get(i) == user_ID):
            similar_users[0] = similarity_data[0].get(i)
            similar_users[1] = similarity_data[1].get(i)
            similar_users[2] = similarity_data[2].get(i)
            similar_users[3] = similarity_data[3].get(i)
            similar_users[4] = similarity_data[4].get(i)
      
    temp = []
    temp2 = []
    
    for user in range(n_recommendations):
        temp += list(rating[rating['userID'] == 'U'+str(int(similar_users[user]))]['placeID'].unique())
        
    temp = list(set(temp))
    
    for index in range(len(temp)):
        temp2 += list(rest_info[rest_info['ID'] == temp[index]]['Avg_Ratings'])
    
    sorted_temp = [x for _,x in sorted(zip(temp2,temp))]
    temp3 = []
    
    print('Recommendations for {}:'.format(user_ID))
    for index in range(1,n_recommendations+1):
        temp3 += list(rest_info[rest_info['ID'] == sorted_temp[-index]]['Cuisine'])[0].split(',')
        temp4 = list(rest_info[rest_info['ID'] == sorted_temp[-index]]['Cuisine'])[0].split(',')
        print("Restaurant: ", sorted_temp[-index])
        print("Cuisines: ", temp4)
    
        
    return

In [372]:
make_recommendation(user_similarity, user_rating, 1001, 5, len(user_info['ID']), rest_info)


Recommendations for 1001:
Restaurant:  135058
Cuisines:  ['Regional']
Restaurant:  135052
Cuisines:  ['Regional', 'Dessert-Ice_Cream']
Restaurant:  135026
Cuisines:  ['Regional', 'Dessert-Ice_Cream']
Restaurant:  132715
Cuisines:  ['Fast_Food']
Restaurant:  132668
Cuisines:  ['Fast_Food']
